# Example 2: Coupled Transmon Qubits

In [ ]:
%load_ext autoreload
%autoreload 2
import time

In [ ]:
import json
import design as d
import names as n

with open("design_variables.json") as in_file:
    initial_design_variables = json.load(in_file)

from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType
from qdesignoptimizer.utils.utils import close_ansys


## Design assembly

In [ ]:
close_ansys()

In [ ]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = True
chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um")
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=chip_type, open_gui=OPEN_GUI
)

n.add_design_variables_to_design(design, initial_design_variables)

In [ ]:
def render_qiskit_metal_design(design, gui):
    d.add_transmon_plus_resonator(design, group=n.GROUP_1)
    d.add_transmon_plus_resonator(design, group=n.GROUP_2)

    d.add_coupler(design)

    d.add_route_interconnects(design)

    d.add_launch_pads(design)

    d.add_chargeline(design, group=n.GROUP_1)
    d.add_chargeline(design, group=n.GROUP_2)

    gui.rebuild()
    gui.autoscale()


render_qiskit_metal_design(design, gui)

## MiniStudies

In [ ]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qdesignoptimizer.design_analysis_types import MeshingMap

# Single resonator

In [ ]:
"""
Here we want to study the following permutations:
- fine mesh conductor
- no fine mesh

single run
14 passes
look at convergence of imaginary part
"""

In [ ]:
def CoupledLineTee_mesh_names(comp_names):
    all_names_to_mesh = [
                        f"prime_cpw_{comp_names}", f"second_cpw_{comp_names}", 
                         ]
    return all_names_to_mesh

In [ ]:
MINI_STUDY_GROUP = n.GROUP_1
MINI_STUDY = ms.get_mini_study_res(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[MINI_STUDY_GROUP],
    opt_target_qubit_freq=False,
    opt_target_qubit_anharm=False,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=True,
    opt_target_resonator_qubit_chi=False,
)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path='out/' + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS,
    meshing_map=[MeshingMap(component_class = CoupledLineTee, mesh_names = CoupledLineTee_mesh_names)]
    
)

group_runs = 1
group_passes = 14
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

### Optimize capacitance target

In [ ]:
MINI_STUDY_BRANCH = 1
MINI_STUDY = ms.get_mini_study_resonator_capacitance(group=MINI_STUDY_BRANCH)
opt_targets = ot.get_opt_target_capacitance(group=MINI_STUDY_BRANCH)

RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CAPACITANCE,
)

In [ ]:
group_runs = 1
group_passes_cap = 10

for i in range(group_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

## View Optimization results

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

In [ ]:
design_analysis.get_eigenmode_results()

In [ ]:
close_ansys()

# Analytical formula for coupling Q

In [ ]:
import numpy as np

In [ ]:
def qc(imp, cap, omega):
    return np.pi / (imp**2 * omega**2 * cap**2)

In [ ]:
freq= 7.590565e9
freq/qc(50, 2.17847e-15, 2*np.pi*freq)/1e3